# 🌍 LLM Translation Agent
This notebook demonstrates an advanced agent for context-aware translations using LLMs.

Features:
- Multiple modes: literal, natural, creative
- Tone control: neutral, formal, casual, professional, poetic
- Translate raw text, URLs, and PDFs
- Option to explain translation choices
- Multi-language translation support
- Interactive widgets for testing

In [ ]:
!pip install openai requests PyPDF2 beautifulsoup4 ipywidgets

In [ ]:

import requests
from bs4 import BeautifulSoup
import PyPDF2
from openai import OpenAI

class LLMTranslationAgent:
    def __init__(self, model="gpt-4o-mini"):
        self.model = model
        self.client = OpenAI()

    def translate(self, text: str, target_lang: str = "French", mode: str = "natural", tone: str = "neutral", explain: bool = False) -> str:
        """
        Translate text using LLM with tone, style, and explanation options.

        :param text: Input text to translate.
        :param target_lang: Target language (e.g., "French", "Spanish").
        :param mode: "literal", "natural", or "creative".
        :param tone: "neutral", "formal", "casual", "professional", "poetic".
        :param explain: If True, provide translation notes.
        """
        instructions = {
            "literal": "Translate word-for-word as closely as possible.",
            "natural": "Translate naturally, as a fluent speaker would write.",
            "creative": "Translate with adaptation, localization, and stylistic creativity."
        }
        prompt = f"""
        Translate the following text into {target_lang}.
        Mode: {mode} — {instructions.get(mode, 'natural translation')}.
        Tone: {tone}.
        { "Also explain translation choices, idioms, or cultural details." if explain else "" }

        Text:
        {text}
        """

        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )
        return resp.choices[0].message.content.strip()

    def translate_url(self, url: str, **kwargs) -> str:
        """Fetch webpage text and translate."""
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        text = " ".join(p.get_text() for p in soup.find_all("p"))
        return self.translate(text, **kwargs)

    def translate_pdf(self, file_path: str, **kwargs) -> str:
        """Extract text from PDF and translate."""
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            text = " ".join(page.extract_text() or "" for page in reader.pages)
        return self.translate(text, **kwargs)

    def multi_translate(self, text: str, target_langs: list, **kwargs) -> dict:
        """Translate into multiple languages at once."""
        results = {}
        for lang in target_langs:
            results[lang] = self.translate(text, target_lang=lang, **kwargs)
        return results


In [ ]:

import ipywidgets as widgets
from IPython.display import display

agent = LLMTranslationAgent()

text_input = widgets.Textarea(value="Hello, how are you?", placeholder="Enter text to translate...", description="Text:", layout=widgets.Layout(width="100%", height="100px"))
target_lang = widgets.Dropdown(options=["French", "Spanish", "German", "Chinese", "Arabic"], value="French", description="Target:")
tone = widgets.Dropdown(options=["neutral", "formal", "casual", "professional", "poetic"], value="neutral", description="Tone:")
mode = widgets.Dropdown(options=["literal", "natural", "creative"], value="natural", description="Mode:")
explain = widgets.Checkbox(value=False, description="Explain translation")

button = widgets.Button(description="Translate", button_style="success")
output = widgets.Output()

def on_translate_clicked(b):
    output.clear_output()
    with output:
        result = agent.translate(text_input.value, target_lang.value, mode.value, tone.value, explain.value)
        print("🔤 Translation:", result)

button.on_click(on_translate_clicked)
display(text_input, target_lang, tone, mode, explain, button, output)


In [ ]:

# Multi-language translation demo
agent = LLMTranslationAgent()
results = agent.multi_translate("The future of AI is promising.", ["French", "Spanish", "Chinese"], mode="natural")
for lang, trans in results.items():
    print(f"{lang}: {trans}")
